# PREPROCES DATA
### 1. Elimina articolele care nu au: id, title, autors, venue, year, keywords, fos, references, n_citation, lang (!=en), abstract sau indexed_abstract
### 2. Pune cate 50000 de articole intr-un fisier


In [1]:
import json
from nltk.corpus import stopwords # pentru eliminare stopwords
import re
import string
from nltk.stem import WordNetLemmatizer # lemmatizes word based on it's parts of speech
import gensim
from nltk.corpus import wordnet # To get words in dictionary with their parts of speech
from collections import Counter 

In [5]:
def elimina_articole_incomplete(fileIn, fileOut):
    fileOut = 'completeData/' + fileOut
    wrongPapers = 0
    okPapers = 0
    i = 0
    r = open(fileIn,'r',encoding='utf-8')
    w = open(fileOut,'w',encoding='utf-8')
    for line in r:
        if i % 1000000 == 0:
            print(i, wrongPapers, okPapers)
        i += 1
        crt_paper = json.loads(line)
        try:
            lang_cp = crt_paper["venue"] 
        except:
            wrongPapers += 1
            continue
        
        okPapers += 1
        w.write(json.dumps(crt_paper))
        w.write('\n')
            
    print(fileIn, wrongPapers, okPapers)
    w.close()
    r.close()

In [8]:
elimina_articole_incomplete('./completeData/dblp_v11_4.json','dblp_v11_5.json')

0 0 0
1000000 2821 997179
2000000 3520 1996480
3000000 7575 2992425
./completeData/dblp_v11_4.json 12693 3257719


## 2

In [9]:
def split_articles():
    r = open('completeData/dblp_v11_5.json','r',encoding='utf-8')
    nr = 1
    current_paper = 1
    w = open('splited_data/file0.txt','w',encoding='utf-8')
    
    for line in r:
        if current_paper % 100000 == 0:
            w.close()
            currentFile = 'splited_data/file' + str(nr) + '.txt'
            nr += 1
            w =  open(currentFile,'w',encoding='utf-8')
        current_paper += 1
        
        crt_paper = json.loads(line)
        w.write(json.dumps(crt_paper))
        w.write('\n')
            
    w.close()
    r.close()

In [10]:
split_articles()

In [14]:
def elimina_punctuatia(word):
    new_word = ""
    for ch in word:
        if ch not in string.punctuation:
            new_word += ch
    new_word = re.sub(r'\d+', '', new_word)
    return new_word

def get_pos( word ):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0]

def is_noun(pos):
    if pos == "n":
        return True
    return False

def is_verb(pos):
    if pos == "v":
        return True
    return False

def lemmatize_word(word, pos):
    wnl = WordNetLemmatizer()
    return (wnl.lemmatize( word, pos ))

def preprocess_indexabs(fileIn,fileOut):
    w = open(fileOut,'w',encoding='utf-8')
    r = open(fileIn,'r',encoding='utf-8')
    for line in r:
        crt_paper = json.loads(line)
        iabstr = crt_paper["indexed_abstract"]["InvertedIndex"] 
        new_iabstr = {}
        for elem in iabstr:
            new_elem = elem.lower()
            new_elem = elimina_punctuatia(new_elem)
            if new_elem in gensim.parsing.preprocessing.STOPWORDS:
                continue
            if len(new_elem) < 3:
                continue
            if not is_verb(get_pos(new_elem)):
                continue
            new_elem = lemmatize_word(new_elem, "v")
            nr_aparitii = len(iabstr[elem])
            try :
                new_iabstr[new_elem] += nr_aparitii
            except:
                new_iabstr[new_elem] = nr_aparitii


        crt_paper["indexed_abstract"]  = new_iabstr 
        w.write(json.dumps(crt_paper))
        w.write('\n')
    r.close()
        
    w.close()

In [15]:
fileIn = 'splited_data/file'
fileOut = 'preprocessedIA/file'
for i in range(0,33):
    fileIn1 = fileIn + str(i) + '.txt'
    fileOut1 = fileOut + str(i) + '.txt'
    preprocess_indexabs(fileIn1,fileOut1 )
    print(fileIn1,fileOut1)

splited_data/file0.txt preprocessedIA/file0.txt
splited_data/file1.txt preprocessedIA/file1.txt
splited_data/file2.txt preprocessedIA/file2.txt
splited_data/file3.txt preprocessedIA/file3.txt
splited_data/file4.txt preprocessedIA/file4.txt
splited_data/file5.txt preprocessedIA/file5.txt
splited_data/file6.txt preprocessedIA/file6.txt
splited_data/file7.txt preprocessedIA/file7.txt
splited_data/file8.txt preprocessedIA/file8.txt
splited_data/file9.txt preprocessedIA/file9.txt
splited_data/file10.txt preprocessedIA/file10.txt
splited_data/file11.txt preprocessedIA/file11.txt
splited_data/file12.txt preprocessedIA/file12.txt
splited_data/file13.txt preprocessedIA/file13.txt
splited_data/file14.txt preprocessedIA/file14.txt
splited_data/file15.txt preprocessedIA/file15.txt
splited_data/file16.txt preprocessedIA/file16.txt
splited_data/file17.txt preprocessedIA/file17.txt
splited_data/file18.txt preprocessedIA/file18.txt
splited_data/file19.txt preprocessedIA/file19.txt
splited_data/file20.

### Grupare cuvinte abstract

In [12]:
def grupare_abstracte(word_dict,fileIn,fileOut):
    #w = open(fileOut,'w',encoding='utf-8')
    r = open(fileIn,'r',encoding='utf-8')
    
    for line in r:
        crt_paper = json.loads(line)
        iabstr = crt_paper["indexed_abstract"]
        
        for elem in iabstr:
            try:
                word_dict[elem] += iabstr[elem]
            except:
                word_dict[elem] = iabstr[elem]

    #w.write(json.dumps(word_dict))
    print(len(word_dict))
    #w.close()
    r.close()
    return word_dict

In [13]:
fileIn = 'preprocessedIA/file'
fileOut = 'IAgroups/dict'
word_dict = {}
for i in range(0,33):
    fileIn1 = fileIn + str(i) + '.txt'
    fileOut1 = fileOut + str(i) + '.txt'
    word_dict = grupare_abstracte(word_dict,fileIn1,fileOut1 )
    print(fileIn1,fileOut1)

4039
preprocessedIA/file0.txt IAgroups/dict0.txt
4519
preprocessedIA/file1.txt IAgroups/dict1.txt
4791
preprocessedIA/file2.txt IAgroups/dict2.txt
4996
preprocessedIA/file3.txt IAgroups/dict3.txt
5152
preprocessedIA/file4.txt IAgroups/dict4.txt
5307
preprocessedIA/file5.txt IAgroups/dict5.txt
5447
preprocessedIA/file6.txt IAgroups/dict6.txt
5590
preprocessedIA/file7.txt IAgroups/dict7.txt
5682
preprocessedIA/file8.txt IAgroups/dict8.txt
5742
preprocessedIA/file9.txt IAgroups/dict9.txt
5814
preprocessedIA/file10.txt IAgroups/dict10.txt
5875
preprocessedIA/file11.txt IAgroups/dict11.txt
5922
preprocessedIA/file12.txt IAgroups/dict12.txt
5965
preprocessedIA/file13.txt IAgroups/dict13.txt
6008
preprocessedIA/file14.txt IAgroups/dict14.txt
6050
preprocessedIA/file15.txt IAgroups/dict15.txt
6089
preprocessedIA/file16.txt IAgroups/dict16.txt
6117
preprocessedIA/file17.txt IAgroups/dict17.txt
6143
preprocessedIA/file18.txt IAgroups/dict18.txt
6167
preprocessedIA/file19.txt IAgroups/dict19.txt


In [2]:
r = open('IAgroups/dict_final.txt','r',encoding='utf-8')
word_dict = json.load(r)
r.close()

In [3]:
word_dict_sorted = sorted((value,key) for (key,value) in word_dict.items())

In [24]:
w = open('IAgroups/dict_final_sorted.txt','w',encoding='utf-8')
w.write(json.dumps(word_dict_sorted))
w.close()

In [4]:
len(word_dict_sorted)

6506

In [5]:
def grupare_abstracte(word_dict,abstracts_list,fileIn):
    r = open(fileIn,'r',encoding='utf-8')
    
    for line in r:
        crt_paper = json.loads(line)
        iabstr = crt_paper["indexed_abstract"]
        current_abstr = []
        for elem in iabstr:
            if word_dict[elem] >= 5 and word_dict[elem] <= 1000:
                current_abstr.append(elem)
        abstracts_list.append(current_abstr)
    r.close()
    return abstracts_list

In [ ]:
fileIn = 'preprocessedIA/file'
abstracts_list = []
for i in range(0,33):
    fileIn1 = fileIn + str(i) + '.txt'
    abstracts_list = grupare_abstracte(word_dict,abstracts_list,fileIn1 )
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [6]:
print(len(abstracts_list))

3257719


In [41]:
print(abstracts_list[0:100])

[['abide'], [], [], ['fractionate'], [], [], [], [], ['solidify'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['redetermine'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['stylise'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['carve', 'credit'], [], ['equilibrate'], [], [], [], [], []]


In [7]:
new_abstracts_list = []
for elem in abstracts_list:
    if elem != []:
        new_abstracts_list.append(elem)

In [8]:
print(len(new_abstracts_list))

54711


In [44]:
print(new_abstracts_list[0:100])

[['abide'], ['fractionate'], ['solidify'], ['redetermine'], ['stylise'], ['carve', 'credit'], ['equilibrate'], ['abridge'], ['incubate'], ['abolish'], ['victimize'], ['succumb'], ['monopolise'], ['proffer'], ['eventuate', 'contest'], ['modernise'], ['elude'], ['creep'], ['disgrace'], ['entice'], ['triple'], ['sedate'], ['allot'], ['major'], ['intrude'], ['fin'], ['irradiate', 'fractionate'], ['occult'], ['stomp'], ['revive'], ['charter'], ['infiltrate'], ['relegate'], ['digitise'], ['itemize'], ['spoil', 'complain'], ['parent'], ['bore'], ['rein'], ['dream'], ['crystallize', 'disengage'], ['prospect'], ['transliterate'], ['miniaturise'], ['await'], ['entice'], ['gentle'], ['redo'], ['oblige'], ['pinch'], ['tout'], ['prod'], ['interpose'], ['assuage'], ['repel'], ['rebind'], ['roleplay'], ['misrepresent'], ['shelter'], ['referee'], ['appoint'], ['puzzle'], ['delocalize'], ['particularize'], ['industrialize'], ['underperform'], ['loaf'], ['peck'], ['relativise'], ['dredge'], ['stumble'],

In [9]:
w = open('abstract_5_100.txt','w',encoding='utf-8')
w.write(json.dumps(new_abstracts_list))
w.close()